In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from nbhelper import *

This notebook presents some exploration and visualisation on two datasets. 

One was collected during the a pointing in VR experiment. 12 participants were asked to reach for a target in VR without real feedback. The end position of the reaching motion is recorded. There are 3 conditions: pointing with the left hand, pointing with the right hand, and pointing the right hand onto the left hand. For each condition, 7 targets indicate the location at which the pointing occurs.

The second experiment is also a pointing experiment in VR but the visual feedback is manipulated so as to create the illusion of several targets being present in the scene. The positive and negative angles until which the illusion is believable are recorded. These are extracted from a series of trials which samples the illusion angles. A sigmoid fit is applied to the data and alpha +/- are reported from the sigmoid fit.

The main idea behind these two datasets is to describe a relationship between the pointing behaviours of participants and their ability to discriminate against optical illusions.

In other words, it is a regression task where we are looking for the relationship:
$$
y = f(x)
$$
where $y$ is the maximum deviation angle for successful optical illusions and $x$ is the user raw pointing performance.

# Read data

In [ ]:
df0 = pd.read_csv('./data/Data1stExpe.csv')

In [ ]:
df0.columns = ['user', 'condition', 'target', 'trial', 'tx', 'ty', 'px', 'py']

In [ ]:
df0.head()

In [ ]:
df1 = pd.read_csv('./data/Data2ndExpe.csv')

In [ ]:
df1.columns = ['user', 'target', 'a-', 'a+']

In [ ]:
df1.head()

# all data from experiment #1

In [ ]:
fig2 = plt.figure(constrained_layout=True, figsize=(12,12))
spec2 = gridspec.GridSpec(ncols=4, nrows=4, figure=fig2)
f2_ax1 = fig2.add_subplot(spec2[3, 0])
f2_ax2 = fig2.add_subplot(spec2[2, 0])
f2_ax3 = fig2.add_subplot(spec2[1, 0])
f2_ax4 = fig2.add_subplot(spec2[0, 0])
f2_ax5 = fig2.add_subplot(spec2[1, 1])
f2_ax6 = fig2.add_subplot(spec2[2, 2])
f2_ax7 = fig2.add_subplot(spec2[3, 3])
axs = [f2_ax1, f2_ax2, f2_ax3, f2_ax4, f2_ax5, f2_ax6, f2_ax7]

for i, (j, grp) in enumerate(df0.groupby(['target', 'condition'])):
    target = j[0]
    condition = j[1]
    
    scatter = axs[target-1].scatter(grp['px'], grp['py'], label='cdt: '+str(grp['condition'].iloc[0]))
    axs[target-1].scatter(grp['tx'].mean(), grp['ty'].mean(), c='k', marker='x')
    confidence_ellipse(grp['px'], grp['py'], axs[target-1], edgecolor=default_colors[condition-1])
    
    axs[target-1].set_title("target {}".format(target))
    
_=axs[0].legend()
_=fig2.suptitle('Per conditions, across targets.')

# Is there a learning effect in the data?

We can wonder whether there is an effect of trial on the pointing behaviour of users, or if we can safely take the mean of end positions.

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(16,4))

for i, grp in df0.groupby(['target', 'condition', 'user']):
    target = i[0]
    condition = i[1]
    
    if (target == 1) and (condition == 1):
        grp.plot(y='px', use_index=False, ax=ax[0], legend=False)
        grp.plot(y='py', use_index=False, ax=ax[1], legend=False)
        grp.plot(x='px', y='py', ax=ax[2], legend=False)

I do not have a definitive answer for this question but it might be useful to think about it. 

# user dependent effects in pointing

Here we compute some summary statistics on the user data. Averaging over (user, condition and target), we compute the mean deviation from the target. We also compute the covariance matrix of the user point cloud, with the variance in both dimensions (vx and vy) and the covariance cxy between x and y.

In [ ]:
# deviation from target
dx = df0.groupby(['target', 'condition', 'user']).apply(lambda x: x['px'].mean() - x['tx'].mean())
dy = df0.groupby(['target', 'condition', 'user']).apply(lambda x: x['py'].mean() - x['ty'].mean())

# dispersion from target
vx = df0.groupby(['target', 'condition', 'user']).apply(lambda x: np.cov(x[['px', 'py']].T)[0,0])
vy = df0.groupby(['target', 'condition', 'user']).apply(lambda x: np.cov(x[['px', 'py']].T)[1,1])
cxy = df0.groupby(['target', 'condition', 'user']).apply(lambda x: np.cov(x[['px', 'py']].T)[0,1])

In [ ]:
data0 = pd.concat([dx, dy, vx, vy, cxy], axis=1)
data0.columns=['dx', 'dy', 'vx', 'vy', 'cxy']
data0 = data0.reset_index()

In [ ]:
data0.head()

In [ ]:
fig2 = plt.figure(constrained_layout=True, figsize=(12,12))
spec2 = gridspec.GridSpec(ncols=4, nrows=4, figure=fig2)
f2_ax1 = fig2.add_subplot(spec2[3, 0])
f2_ax2 = fig2.add_subplot(spec2[2, 0])
f2_ax3 = fig2.add_subplot(spec2[1, 0])
f2_ax4 = fig2.add_subplot(spec2[0, 0])
f2_ax5 = fig2.add_subplot(spec2[1, 1])
f2_ax6 = fig2.add_subplot(spec2[2, 2])
f2_ax7 = fig2.add_subplot(spec2[3, 3])
axs = [f2_ax1, f2_ax2, f2_ax3, f2_ax4, f2_ax5, f2_ax6, f2_ax7]

for i, (j, grp) in enumerate(data0.groupby(['target', 'condition'])):
    target = j[0]
    condition = j[1]
    
    scatter = axs[target-1].scatter(grp['dx'], grp['dy'], label=grp['condition'].iloc[0])
    axs[target-1].scatter(0, 0, c='k', marker='x')
    
    color = scatter.to_rgba(0)
    confidence_ellipse(grp['dx'], grp['dy'], axs[target-1], edgecolor=default_colors[condition-1])
    
    axs[target-1].set_title("target {}".format(target))

    
axs[0].legend()
_=fig2.suptitle('Per conditions, across targets - deviation means from target.')

In [ ]:
fig2 = plt.figure(constrained_layout=True, figsize=(12,12))
spec2 = gridspec.GridSpec(ncols=4, nrows=4, figure=fig2)
f2_ax1 = fig2.add_subplot(spec2[3, 0])
f2_ax2 = fig2.add_subplot(spec2[2, 0])
f2_ax3 = fig2.add_subplot(spec2[1, 0])
f2_ax4 = fig2.add_subplot(spec2[0, 0])
f2_ax5 = fig2.add_subplot(spec2[1, 1])
f2_ax6 = fig2.add_subplot(spec2[2, 2])
f2_ax7 = fig2.add_subplot(spec2[3, 3])
axs = [f2_ax1, f2_ax2, f2_ax3, f2_ax4, f2_ax5, f2_ax6, f2_ax7]

for i, (j, grp) in enumerate(data0.groupby(['target', 'condition'])):
    target = j[0]
    condition = j[1]
    
    if condition == 1:
        sns.scatterplot(data=grp, x='dx', y='dy', hue='user', ax=axs[target-1], legend=False, palette='tab10')
        axs[target-1].set_title("target {}".format(target))

_=fig2.suptitle('For one condition, across targets, deviation means from target with user colors.')

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(12,16))

for i, (j, grp) in enumerate(data0.groupby(['user', 'condition'])):
    user = j[0]
    condition = j[1]
    
    tmp = select(grp, target=[1,2,3,4])
    tmp.plot(x='dx', y='dy', ax=ax[condition-1, 0], label=user)

    tmp = select(grp, target=[3,5,6,7])
    tmp.plot(x='dx', y='dy', ax=ax[condition-1, 1], label=user)

    ax[condition-1, 0].set_title("target [1,2,3,4]")
    ax[condition-1, 1].set_title("target [3,5,6,7]")
    ax[condition-1, 0].set_ylabel("condition {}".format(condition))

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(12,16))

for i, (j, grp) in enumerate(data0.groupby(['user', 'condition'])):
    user = j[0]
    condition = j[1]
    
    tmp = select(grp, target=[1,2,3,4])
    tmp.plot(x='vx', y='vy', ax=ax[condition-1, 0], label=user)

    tmp = select(grp, target=[3,5,6,7])
    tmp.plot(x='vx', y='vy', ax=ax[condition-1, 1], label=user)

    ax[condition-1, 0].set_title("target [1,2,3,4]")
    ax[condition-1, 1].set_title("target [3,5,6,7]")
    ax[condition-1, 0].set_ylabel("condition {}".format(condition))

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(12,16))

for i, (j, grp) in enumerate(data0.groupby(['user', 'condition'])):
    user = j[0]
    condition = j[1]
    
    tmp = select(grp, target=[1,2,3,4])
    tmp.plot(y='cxy', ax=ax[condition-1, 0], label=user, use_index=False)

    tmp = select(grp, target=[3,5,6,7])
    tmp.plot(y='cxy', ax=ax[condition-1, 1], label=user, use_index=False)

    ax[condition-1, 0].set_title("target [1,2,3,4]")
    ax[condition-1, 1].set_title("target [3,5,6,7]")
    ax[condition-1, 0].set_ylabel("condition {}".format(condition))

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def plot_user_target_distributions(user=i):
    fig2 = plt.figure(constrained_layout=True, figsize=(12,12))
    spec2 = gridspec.GridSpec(ncols=4, nrows=4, figure=fig2)
    f2_ax1 = fig2.add_subplot(spec2[3, 0])
    f2_ax2 = fig2.add_subplot(spec2[2, 0])
    f2_ax3 = fig2.add_subplot(spec2[1, 0])
    f2_ax4 = fig2.add_subplot(spec2[0, 0])
    f2_ax5 = fig2.add_subplot(spec2[1, 1])
    f2_ax6 = fig2.add_subplot(spec2[2, 2])
    f2_ax7 = fig2.add_subplot(spec2[3, 3])
    axs = [f2_ax1, f2_ax2, f2_ax3, f2_ax4, f2_ax5, f2_ax6, f2_ax7]

    for i, (j, grp) in enumerate(df0.groupby(['target', 'condition', 'user'])):
        target = j[0]
        condition = j[1]
        cur_user = j[2]

        if (cur_user == user):        
            scatter = axs[target-1].scatter(grp['px'], grp['py'], label=grp['condition'].iloc[0])
            axs[target-1].scatter(grp['tx'].mean(), grp['ty'].mean(), c='k', marker='x')
            confidence_ellipse(grp['px'], grp['py'], axs[target-1], edgecolor=default_colors[condition-1])

    axs[0].legend()
    fig.suptitle("user {}".format(user))
#     fig.show()

In [ ]:
slider = widgets.IntSlider(min=1, max=12, step=1, value=1)
_=interact(plot_user_target_distributions, user=slider)

In [ ]:
select(data0, condition=3)['vx'].hist()
select(data0, condition=3)['vy'].hist()

# Plot data experiment 2

In [ ]:
df1.head()

In [ ]:
df1['da'] = df1['a+'] - df1['a-']

In [ ]:
fig, axs = plt.subplots(1,4, figsize=(16,4))
axs = axs.flatten()

# for i, grp in df1.groupby('target'):
#     ax = axs[i-1]
#     grp['-a-'] = -grp['a-']
#     grp.plot(x='user', y='-a-', ax=ax)

for i, grp in df1.groupby('target'):
    ax = axs[i-1]
    grp.plot(x='user', y='a-', ax=ax)
    
for i, grp in df1.groupby('target'):
    ax = axs[i-1]
    grp.plot(x='user', y='a+', ax=ax)
    
for i, grp in df1.groupby('target'):
    ax = axs[i-1]
    grp.plot(x='user', y='da', ax=ax)
    ax.set_title("target {}".format(i))

We find that a+ and a- seem to be correlated across users and targets.

In [ ]:
fig, ax = plt.subplots()

for i, grp in df1.groupby('target'):
    grp.plot(x='user', y='da', ax=ax, label=i)
    ax.set_title("all targets")

This picture makes me feel that the value for alpha is independent of the target. In other words, the position at which the illusion occurs does not play a role on its validity - in the conditions of the experiment.

## Distributions of da

It can be interesting to investigate the distribution of da, as this will be the variable we want to predict from the pointing behaviour of users. 

In [ ]:
df1['da'].hist()

It seems to be normally distributed in log space.

In [ ]:
df1['exp_da'] = np.exp(df1['da'])

In [ ]:
df1['exp_da'].hist()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))

# target on alpha
means = df1.groupby('target')[['a-', 'a+']].mean()
stds = df1.groupby('target')[['a-', 'a+']].std()

means.plot(ax=ax[0])
ax[0].fill_between(np.arange(1,5), means['a-']-stds['a-'], means['a-']+stds['a-'], color='k', alpha=0.1)
ax[0].fill_between(np.arange(1,5), means['a+']-stds['a+'], means['a+']+stds['a+'], color='k', alpha=0.1)
ax[0].set_xticks(np.arange(1,5))
ax[0].grid()

# user on alpha
means = df1.groupby('user')[['a-', 'a+']].mean()
stds = df1.groupby('user')[['a-', 'a+']].std()

means.plot(ax=ax[1])
ax[1].fill_between(np.arange(1,13), means['a-']-stds['a-'], means['a-']+stds['a-'], color='k', alpha=0.1)
ax[1].fill_between(np.arange(1,13), means['a+']-stds['a+'], means['a+']+stds['a+'], color='k', alpha=0.1)
ax[1].set_xticks(np.arange(1,13))
ax[1].grid()

There does not seem to be a clear effect of target on alpha, as the mean over user is very noisy. There is however a strong user dependence on the value of alpha.

# relation between pointing performance and alpha?

Given the structure of the data in terms of dependence of alpha against target and user, it seems more interesting to attempt to model the relationship between alpha and users.

In [ ]:
a = df1.groupby('user').mean()['da']
b = select(data0, condition=3).groupby('user').mean()[['dx', 'dy']].apply(np.linalg.norm, axis=1)
plt.scatter(b, a)

This is a naive plot, but it shows a potentially interesting relation: the lower the pointing deviation, the lower is alpha. As the pointing deviation increases, the likelihood of alpha increases too. We also have a bigger spread for alpha, as the pointing deviation increases. It is like a "trumpet".

In [ ]:
a = df1.groupby('user').mean()['da']
c = select(data0, condition=1).groupby('user').mean()['vx']
d = select(data0, condition=3).groupby('user').mean()['vx']

fig, ax = plt.subplots()
ax.scatter(d/c, a)

If we check the ratio of the variance across x, we find this curve.

In [ ]:
c = select(data0, condition=1).groupby('user').mean()[['dx', 'dy']].apply(np.linalg.norm, axis=1)

In [ ]:
d = select(data0, condition=2).groupby('user').mean()[['dx', 'dy']].apply(np.linalg.norm, axis=1)

# predict da from (dx, dy, vx, vy, cxy) in conditions 1,2,3

To do so, we will first create our X and y vectors.

In [ ]:
data0.head()

In [ ]:
X = data0.set_index(['target', 'user', 'condition']).unstack()

In [ ]:
X = X.reset_index()

In [ ]:
# this removes the multiindex from columns, and creates a simpler single index
new_cols = [str(i)+str(j) for (i,j) in pd.Index(X.columns._values, tupleize_cols=False)]

In [ ]:
X.columns = new_cols

In [ ]:
X.head()

In [ ]:
df1.head()

In [ ]:
y = X[['target', 'user']].copy()
y['da'] = pd.Series()
y = y.set_index(['target', 'user'])

## a. with targets repeated: 7 targets * 12 users

Here, we consider that measurements in experiment 1 and targets [1,2,3,4] are related to the measurement of target 1 in experiment 2, while 5, 6 and 7 are linked to 2, 3 and 4, respectively.

In [ ]:
for i, row in X.iterrows():
    user = row['user']
    target = row['target']
    
    target_map = {1:1, 2:1, 3:1, 4:1, 5:2, 6:3, 7:4}
    y.loc[target, user] = df1.set_index(['target', 'user']).loc[target_map[target], user]['da']

In [ ]:
y = y.reset_index()

In [ ]:
y.head()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)

In [ ]:
inputs = keras.Input(shape=(15,))
x = layers.Dense(15, activation='relu')(inputs)
x = layers.Dense(15, activation='relu')(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mse', 
              optimizer=keras.optimizers.SGD())

In [ ]:
history = model.fit(X.iloc[:, 2:], y['da'].values, batch_size=16, epochs=200, validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
y_pred = model.predict(X.iloc[:, 2:])

In [ ]:
plt.plot(y['da'])
plt.plot(y_pred)

## b. with only 4 targets

In [ ]:
X4 = select(X, target=[3,5,6,7]).iloc[:, 2:]

In [ ]:
y4 = select(y, target=[3,5,6,7])['da'].values

In [ ]:
inputs = keras.Input(shape=(15,))
x = layers.Dense(17, activation='relu')(inputs)
x = layers.Dense(17, activation='relu')(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mse', 
              optimizer=keras.optimizers.SGD())

In [ ]:
history = model.fit(X4, y4, batch_size=16, epochs=200, validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
y4_pred = model.predict(X4)

In [ ]:
plt.plot(y4)
plt.plot(y4_pred)

# check model generalisation with cross-validation

In [ ]:
from sklearn.model_selection import KFold, RepeatedKFold

In [ ]:
kf = KFold(n_splits=12)
rkf = RepeatedKFold(n_splits=4, n_repeats=3, random_state=1234)

In [ ]:
for train, test in kf.split(np.arange(1,13)):
    print("%s %s" % (train, test))

In [ ]:
for train, test in rkf.split(np.arange(1,13)):
    print("%s %s" % (train, test))

In [ ]:
def get_model():
    inputs = keras.Input(shape=(15,))
    x = layers.Dense(4, activation='relu')(inputs)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    model.compile(loss='mse', optimizer=keras.optimizers.SGD())
    return model

In [ ]:
res = []

for i, (train, test) in enumerate(rkf.split(np.arange(1,13))):
    print("{}: {} {}".format(i, train, test))

    model = get_model()
    
    grouped = X.groupby('user')
    x_train = pd.concat([grouped.get_group(i+1) for i in train]).iloc[:, 2:]
    x_test = pd.concat([grouped.get_group(i+1) for i in test]).iloc[:, 2:]
    grouped = y.groupby('user')
    y_train = pd.concat([grouped.get_group(i+1) for i in train])['da'].values
    y_test = pd.concat([grouped.get_group(i+1) for i in test])['da'].values
    
    history = model.fit(x_train, y_train, batch_size=2, epochs=25, validation_split=0.2, verbose=0)
    y_pred = model.predict(x_test)
    loss = model.evaluate(x_test, y_test)
    res.append((loss, y_test, y_pred))

In [ ]:
loss = [tmp[0] for tmp in res]
y_test = np.array([tmp[1] for tmp in res])
y_pred = np.array([tmp[2] for tmp in res])

In [ ]:
plt.plot(loss)

In [ ]:
plt.plot(y_test[7])
plt.plot(y_pred[7])

In [ ]:
plt.hist(y['da'])
plt.hist(y_pred.reshape(-1), alpha=0.2)

In [ ]:
plt.hist(y_pred.reshape(-1))

In [ ]:
model = get_model()

In [ ]:
model = get_model()

model.fit(x_train, y_train, batch_size=16, epochs=25, validation_split=0.2)

model.evaluate(x_test, y_test)

In [ ]:
X0.shape

In [ ]:
x_train, x_test = X.iloc[:50, 2:], X.iloc[50:, 2:]
y_train, y_test = y['da'].values[:50], y['da'].values[50:]

In [ ]:
X0 = X.iloc[0, 2:].values

In [ ]:
X0.reshape(1,15)

In [ ]:
model.predict(X0.reshape(1,15))

In [ ]:
ypred = model.predict(X.iloc[:, 2:])

In [ ]:
y['da']

In [ ]:
pd.DataFrame(ypred)

In [ ]:
y['da'].hist()

In [ ]:
(ypred.reshape(-1) - y['da']).mean()

In [ ]:
(ypred.reshape(-1) - y['da']).std()

# using GPs